In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
from pathlib import Path

#torch 
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [2]:
module_path = str(Path.cwd().parents[0].parents[0] / "methods")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [3]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

# print(bhArray.shape)
# print(sphArray.shape)

In [4]:
#Kombinerer dataene for å kunne kjøre gjennom modellen på et samlet datasett
dataArray = np.concatenate((bhArray,sphArray),axis=0)

# np.shape(dataArray)

In [5]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [7]:
#Split data 75% i train og 25% i test
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [8]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [9]:
#Transformere numpy til tensor
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [10]:
#Sette data til å vere dataloaders
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

In [11]:
# loop through the dataloader and print out each batch of data
# for batch_idx, (data_batch, target_batch) in enumerate(trainLoader):
#     print(f"Batch {batch_idx}:")
#     print(f"Data: {data_batch}")
#     print(f"Targets: {target_batch}")

In [12]:
#Define the ConvModel class
class ConvModel(nn.Module):
    #Constructor that initializes the layers of the model
    def __init__(self, dropout):

        super(ConvModel, self).__init__()
        #opprette conv lag. Bildene har 3 lag 
        
        #Dette funker ikke rør
        #Define convolutional layers for the input image
        #Input image has 3 challes, 3 layers
        #The first convolutional layer has 16 output channels and uses a kernel size of 3x3
        #ANd padding is set to 0
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=0)
        #kanskje ha flere layers?

        #Unchange
        #Define fully connected layers
        #Input size to the first fully connected layer is 3*3*256
        #The first fully connected layer has 128 output units
        #The secon fully connected layer has 2 output units
        self.fc1 = nn.Linear(3*3*256, 128)
        self.fc2 = nn.Linear(128,2)

        #Define the dropout layer to prevent overfitting
        self.dropout = nn.Dropout(dropout)


    #Enten her eller over skal komme endringane for convolutional layer
    def forward(self, x:Tensor):
        
        #x = F.conv2d(x, self.conv1_filter, bias=self.conv1.bias, stride=1, padding=1)
        #Apply first conv layer
        x= self.conv1(x)
        #Apply relu activation function
        x = F.relu(x) 
        #Apply max pooling with a kernel size 2
        x = F.max_pool2d(x,2)
        #Apply the second conv layer
        x = self.conv2(x)
        #Apply relu activation function
        x = F.relu(x)
        #Apply max pooling with a kernel size of 2
        x = F.max_pool2d(x,2)
        #Apply the third conv layer
        x = self.conv3(x)
        #Apply relu activation function
        x = F.relu(x)
        #Appply max pooling with a kernel size 3
        x = F.max_pool2d(x,3)
        #Flatten the output of the conv layers
        x = torch.flatten(x, 1)
        
        #Apply the first fully connected layer
        x = self.fc1(x)
        #Apply relu activation function
        x = F.relu(x)
        x = self.dropout(x)
        #Apply teh second fully connected layer
        x = self.fc2(x)
        x = self.dropout(x)
        #Apply droput to layers??
        return x

In [13]:
from torchsummary import summary

#Her måp det endres sidan den aksepterer ikke parametere gitt
model = ConvModel(0).to(device)
summary(model, (3, 50, 50))

ModuleNotFoundError: No module named 'torchsummary'

In [ ]:
def train(model, train_loader, test_loader, optimizer, criterion, n_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    #Start epoch
    for epoch in range(n_epochs):
        # Train
        model.train()
        train_loss = 0
        
        correct = 0
        total = 0
        
        
        #Train loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            optimizer.zero_grad()
            outputs = model(inputs.permute(0, 3, 1, 2)).float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        #Train loop end
        

        train_acc = 100 * correct / total
        train_loss /= len(train_loader)

        # Test
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device).float(), labels.to(device).long()
                outputs = model(inputs.permute(0, 3, 1, 2)).float()
                loss = criterion(outputs, labels)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        test_acc = 100 * correct / total
        test_loss /= len(test_loader)

        # Print results
        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

In [ ]:
model = ConvModel(dropout=0.5)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train(model, trainLoader, testLoader, optimizer, criterion, n_epochs=10)

Epoch 1/10
Train Loss: 0.7604, Train Accuracy: 66.08%
Test Loss: 0.4517, Test Accuracy: 78.24%
Epoch 2/10
Train Loss: 0.5026, Train Accuracy: 73.61%
Test Loss: 0.3925, Test Accuracy: 83.64%
Epoch 3/10
Train Loss: 0.4681, Train Accuracy: 75.15%
Test Loss: 0.3734, Test Accuracy: 83.61%
Epoch 4/10
Train Loss: 0.4399, Train Accuracy: 76.96%
Test Loss: 0.3304, Test Accuracy: 85.65%
Epoch 5/10
Train Loss: 0.4272, Train Accuracy: 77.28%
Test Loss: 0.3357, Test Accuracy: 85.67%
Epoch 6/10
Train Loss: 0.4212, Train Accuracy: 77.30%
Test Loss: 0.3672, Test Accuracy: 84.45%
Epoch 7/10
Train Loss: 0.4082, Train Accuracy: 77.76%
Test Loss: 0.3762, Test Accuracy: 82.57%
Epoch 8/10
Train Loss: 0.3999, Train Accuracy: 78.57%
Test Loss: 0.2986, Test Accuracy: 87.56%
Epoch 9/10
Train Loss: 0.4019, Train Accuracy: 78.55%
Test Loss: 0.3058, Test Accuracy: 87.17%
Epoch 10/10
Train Loss: 0.3914, Train Accuracy: 78.88%
Test Loss: 0.3074, Test Accuracy: 87.45%
